<a href="https://colab.research.google.com/github/joshiayush/cardmatch/blob/main/notebooks/card_seek_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain >/dev/null
!pip install langchain-community >/dev/null
!pip install langchain-openai >/dev/null
!pip install pypdf >/dev/null
!pip install chromadb >/dev/null
!pip install pymongo >/dev/null

In [2]:
import os
import getpass

In [3]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN_API_KEY:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

LANGCHAIN_API_KEY:··········
OPENAI_API_KEY:··········


In [4]:
MONGO_URI = getpass.getpass("MONGO_URI:")

MONGO_URI:··········


In [5]:
import pathlib
import re
from typing import Dict, Any, Optional

import numpy as np
import pandas as pd
from pymongo import MongoClient
from langchain.schema import (
    Document,
    BaseRetriever,
    messages_from_dict,
    messages_to_dict,
)
from langchain.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredEPubLoader,
    WebBaseLoader,
)
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.document_compressors.embeddings_filter import EmbeddingsFilter
from langchain.memory.chat_message_histories import MongoDBChatMessageHistory

In [8]:
_cc_sheets = ["sbi", "axis", "hdfc"]


def load_cc_info() -> Dict:
    cc_info = dict()
    for sheet in _cc_sheets:
        df = pd.read_excel("cc_urls.xlsx", sheet_name=sheet)
        cc_info.update(df.set_index("card_name").T.to_dict())
    return cc_info

In [9]:
credit_cards_info = load_cc_info()

<ipython-input-8-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())
<ipython-input-8-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())


In [10]:
class DocumentLoaderException(Exception):
    pass

In [11]:
class EPubReader(UnstructuredEPubLoader):
    def __init__(self, file_path: str | list[str], **kwargs: Any):
        super().__init__(file_path, **kwargs, mode="elements", strategy="fast")

In [12]:
class DocumentLoader(object):
    """Loads in a document with a supported extension."""

    supported_extensions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EPubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader,
    }

In [13]:
def load_document(filepath: str) -> list[Document]:
  """Load a file and return it as a list of Document."""
  ext = pathlib.Path(filepath).suffix

  loader = DocumentLoader.supported_extensions.get(ext)
  if not loader:
    raise DocumentLoaderException(f"Unsupported file extension: {ext}")

  return loader(filepath).load()

In [14]:
def configure_retriever(
    docs: list[Document], use_compression: Optional[bool] = False
) -> BaseRetriever:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800, chunk_overlap=200, separators=["\n\n## ", "\n\n", "\n", ". "]
    )
    splits = text_splitter.split_documents(docs)

    for i, split in enumerate(splits):
        split.metadata["section_id"] = i // 10

    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(splits, embeddings)

    base_retriever = vectordb.as_retriever(
        search_type="mmr", search_kwargs={"k": 6, "fetch_k": 20, "alpha": 0.6}
    )
    retriever = MultiQueryRetriever.from_llm(
        llm=ChatOpenAI(temperature=0), retriever=base_retriever
    )
    if use_compression:
        embeddings_filter = EmbeddingsFilter(
            embeddings=embeddings, similarity_threshold=0.76
        )
        return ContextualCompressionRetriever(
            base_compressor=embeddings_filter, base_retriever=retriever
        )
    return retriever

In [15]:
DB_NAME = "chatbot_db"
COLLECTION_NAME = "chat_history"

In [29]:
custom_prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template="""
You are a helpful AI assistant.

Given the following:
- **Chat History:**
  {chat_history}
- **Retrieved Context:**
  {context}
- **User's Latest Question:**
  {question}

Generate a response that maintains conversation context and utilizes relevant retrieved information.
"""
)

In [30]:
def configure_chain(retriever: BaseRetriever) -> Chain:
    """Configure chain with a retriever."""
    chat_memory = MongoDBChatMessageHistory(
        connection_string=MONGO_URI,
        database_name=DB_NAME,
        collection_name=COLLECTION_NAME,
        session_id="test_session",
    )
    memory = ConversationBufferMemory(
        memory_key="chat_history", chat_memory=chat_memory, return_messages=True
    )
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    return ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": custom_prompt},
        verbose=True,
    )

In [31]:
def configure_qa_chain(cc_info: Dict) -> Chain:
    """Read documents, configure retriever, and the chain."""
    docs = []
    for key, info in cc_info.items():
        if info["card_link"] is np.nan:
            continue

        card_doc = WebBaseLoader(info["card_link"]).load()
        docs.extend(card_doc)

        tnc_doc = load_document(info["tnc"])
        docs.extend(tnc_doc)
        break

    retriever = configure_retriever(docs, use_compression=True)
    return configure_chain(retriever)

In [32]:
qa_chain = configure_qa_chain(credit_cards_info)

In [25]:
qa_chain.run("What's in the chat history")

<ipython-input-25-e94ed29edd5d>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run("What's in the chat history")




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "ELITE Master Card Aw_Rose Gold."
Human: What is the joining fee of this credit card?
Assistant: The joining fee for the SBI Card ELITE is Rs. 4,999 plus taxes. However, there is no specific mention of an "ELITE Master Card Aw_Rose Gold" credit card in the provided context.
Human: What benefits do I get in movie tickets every month?
Assistant: With the SBI Card ELITE, you can avail a discount on movie tickets every month. The total amount payable for the tickets would be reduced by Rs. 500 or the cost of 2 tickets, whichever is less. This discount is applicable for a maximum of two tickets booked in a month. Note that you will still need to pay the convenience fee as levied

'It seems there was a misunderstanding earlier. I do have access to the current chat session, which includes our ongoing conversation. Here\'s a summary of what we\'ve discussed so far:\n\n1. You asked about the full name of a credit card, and I provided the name "ELITE Master Card Aw_Rose Gold."\n2. You inquired about the joining fee for this card, and I mentioned the fee for the SBI Card ELITE, noting that there wasn\'t specific information about an "ELITE Master Card Aw_Rose Gold."\n3. You asked about the benefits related to movie tickets, and I explained the discount available with the SBI Card ELITE on movie tickets booked through BookMyShow.\n\nIf you have any more questions or need further clarification, feel free to ask!'

In [26]:
qa_chain.run("What is the full name of the credit card?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "ELITE Master Card Aw_Rose Gold."
Human: What is the joining fee of this credit card?
Assistant: The joining fee for the SBI Card ELITE is Rs. 4,999 plus taxes. However, there is no specific mention of an "ELITE Master Card Aw_Rose Gold" credit card in the provided context.
Human: What benefits do I get in movie tickets every month?
Assistant: With the SBI Card ELITE, you can avail a discount on movie tickets every month. The total amount payable for the tickets would be reduced by Rs. 500 or the cost of 2 tickets, whichever is less. This discount is applicable for a maximum of two tickets booked in a month. Note that you will still need to pay the convenience fee as levied

'The full name of the credit card referred to as "ELITE Master Card Aw_Rose Gold" seems to be a variation or specific design of the SBI Card ELITE. The retrieved context mentions the "ELITE Master Card Aw_Rose Gold" as a design on the face of the card, but the benefits and fees align with those of the SBI Card ELITE. If you have any more questions or need further clarification about the card, feel free to ask!'

In [27]:
qa_chain.run("What is the joining fee of this credit card?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "ELITE Master Card Aw_Rose Gold."
Human: What is the joining fee of this credit card?
Assistant: The joining fee for the SBI Card ELITE is Rs. 4,999 plus taxes. However, there is no specific mention of an "ELITE Master Card Aw_Rose Gold" credit card in the provided context.
Human: What benefits do I get in movie tickets every month?
Assistant: With the SBI Card ELITE, you can avail a discount on movie tickets every month. The total amount payable for the tickets would be reduced by Rs. 500 or the cost of 2 tickets, whichever is less. This discount is applicable for a maximum of two tickets booked in a month. Note that you will still need to pay the convenience fee as levied

'The joining fee for the ELITE Master Card Aw_Rose Gold, which appears to be a variation or specific design of the SBI Card ELITE, is Rs. 4,999 plus taxes. This fee aligns with the standard joining fee for the SBI Card ELITE. If you have any more questions or need further clarification about the card, feel free to ask!'

In [28]:
qa_chain.run("What benefits do I get in movie tickets every month?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "ELITE Master Card Aw_Rose Gold."
Human: What is the joining fee of this credit card?
Assistant: The joining fee for the SBI Card ELITE is Rs. 4,999 plus taxes. However, there is no specific mention of an "ELITE Master Card Aw_Rose Gold" credit card in the provided context.
Human: What benefits do I get in movie tickets every month?
Assistant: With the SBI Card ELITE, you can avail a discount on movie tickets every month. The total amount payable for the tickets would be reduced by Rs. 500 or the cost of 2 tickets, whichever is less. This discount is applicable for a maximum of two tickets booked in a month. Note that you will still need to pay the convenience fee as levied

'The ELITE Master Card Aw_Rose Gold, which appears to be a variation of the SBI Card ELITE, offers monthly movie ticket benefits. You can enjoy complimentary movie tickets worth Rs. 6,000 every year. Each month, you can avail a discount on movie tickets, with a maximum discount of Rs. 250 per ticket for up to two tickets. This offer is valid for transactions involving at least two tickets per booking per month. Please note that the convenience fee will still be applicable. To redeem this offer, you need to book your tickets through the www.bookmyshow.com website and follow the specific steps to apply the discount using your SBI Card ELITE.'